In [2]:
import pandas as pd
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline 
sns.set(color_codes=True)
import warnings
warnings.filterwarnings('ignore')

# **- Lets go through the first few rows of our dataset.**

In [2]:
df = pd.read_csv('/content/drive/My Drive/Provider_Info.csv')
# To display the top 5 rows
df.head(5)

,Unnamed: 0,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider County Name,Number of Certified Beds,Average Number of Residents Per Day,Provider Type,Overall Rating,Health Inspection Rating,QM Rating,Staffing Rating,Reported Total Nurse Staffing Hours per Resident per Day,Reported Physical Therapist Staffing Hours per Resident Per Day,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Number of Penalties,Avg Hospital Stay,Treatment cost (in thousands $),Hospital readmission rates(in %),Patient wait time (mins),Patient satisfaction rate(out of 10),Patient safety(out of 10),ER wait time,Primary Care Provider_PCP
0,0,365333,BOWLING GREEN MANOR,1021 W POE RD,BOWLING GREEN,OH,43402,4193524694,Wood,92,78.0,Medicare and Medicaid,4,3,5,3,3.45472,0.03404,40.667,0,4,0,0,3,20,32,32,4,6.0,4,No
1,1,315254,ALARIS HEALTH AT BOULEVARD EAST,6819 BOULEVARD EAST,GUTTENBERG,NJ,7093,2018683600,Hudson,108,96.0,Medicare and Medicaid,5,5,4,5,NaN,0.05574,3.333,0,0,0,0,5,4,42,15,5,7.0,5,FP
2,2,325071,AZTEC HEALTHCARE,500 CARE LANE,AZTEC,NM,87410,5053349445,San Juan,88,75.7,Medicare and Medicaid,2,2,4,3,3.23398,0.12765,104.000,1,4,0,0,3,14,36,46,3,NaN,2,FP
3,3,265873,UNION CARE CENTER,1080 MARIE LANE,UNION,MO,63084,6362068585,Franklin,60,49.9,Medicare and Medicaid,5,5,4,3,3.10734,0.02423,14.400,0,0,0,0,4,5,21,32,6,NaN,4,GP
4,4,495309,PELICAN HEALTH NORFOLK,827 NORVIEW AVENUE,NORFOLK,VA,23509,7578536281,Norfolk City,60,49.3,Medicare and Medicaid,2,2,4,2,3.17798,NaN,82.667,0,18,0,0,9,34,25,82,5,5.0,5,FP


## **Lets see what are the columns and see how many columns are there**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12913 entries, 0 to 12912
Data columns (total 31 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Unnamed: 0                                                       12913 non-null  int64  
 1   Federal Provider Number                                          12913 non-null  object 
 2   Provider Name                                                    12913 non-null  object 
 3   Provider Address                                                 12913 non-null  object 
 4   Provider City                                                    12913 non-null  object 
 5   Provider State                                                   12913 non-null  object 
 6   Provider Zip Code                                                12913 non-null  int64  
 7   Provider Phone Number                   

## **Let's start our cleaning process.**


Lets remove all the columns like Provider Name,Address,Zip Code and other similar columns which provides no useful contribution for the predictive

In [4]:
df.drop(df.iloc[:, 0:9], inplace = True, axis = 1) 
df.head()

,Number of Certified Beds,Average Number of Residents Per Day,Provider Type,Overall Rating,Health Inspection Rating,QM Rating,Staffing Rating,Reported Total Nurse Staffing Hours per Resident per Day,Reported Physical Therapist Staffing Hours per Resident Per Day,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Number of Penalties,Avg Hospital Stay,Treatment cost (in thousands $),Hospital readmission rates(in %),Patient wait time (mins),Patient satisfaction rate(out of 10),Patient safety(out of 10),ER wait time,Primary Care Provider_PCP
0,92,78.0,Medicare and Medicaid,4,3,5,3,3.45472,0.03404,40.667,0,4,0,0,3,20,32,32,4,6.0,4,No
1,108,96.0,Medicare and Medicaid,5,5,4,5,NaN,0.05574,3.333,0,0,0,0,5,4,42,15,5,7.0,5,FP
2,88,75.7,Medicare and Medicaid,2,2,4,3,3.23398,0.12765,104.000,1,4,0,0,3,14,36,46,3,NaN,2,FP
3,60,49.9,Medicare and Medicaid,5,5,4,3,3.10734,0.02423,14.400,0,0,0,0,4,5,21,32,6,NaN,4,GP
4,60,49.3,Medicare and Medicaid,2,2,4,2,3.17798,NaN,82.667,0,18,0,0,9,34,25,82,5,5.0,5,FP


**Checking how many rows are empty for each column.**

In [5]:
df.isnull().sum()


Number of Certified Beds                                              0
Average Number of Residents Per Day                                   0
Provider Type                                                         0
Overall Rating                                                        0
Health Inspection Rating                                              0
QM Rating                                                             0
Staffing Rating                                                       0
Reported Total Nurse Staffing Hours per Resident per Day            497
Reported Physical Therapist Staffing Hours per Resident Per Day     553
Total Weighted Health Survey Score                                  626
Number of Facility Reported Incidents                                 0
Number of Substantiated Complaints                                    0
Number of Fines                                                       0
Total Number of Penalties                                       

 We see that three columns
- Reported Total Nurse Staffing Hours per Resident per Day            
- Reported Physical Therapist Staffing Hours per Resident Per Day    
- Total Weighted Health Survey Score
are having nearly ~500 missing values from 12913 rows , so we can impute the data instead of removing the data.

Whereas in the case of Patient safety(out of 10) we see that nearly 1/3rd of the data is missing and it makes no sense to impute the data hence we can remove the columns


In [6]:
df.drop('Patient safety(out of 10)', axis=1, inplace=True)

Hence we have removed the Patient Safety column and now we will be imputing the data for the other 3 columns.

Before Imputing lets just encode all our Data
We will be encoding two columns
- Provider Type 
- Primary_Care_Provider

In [7]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse=False)
data_encoded = pd.DataFrame (encoder.fit_transform(df[['Provider Type']]))
data_encoded.columns = encoder.get_feature_names(['Provider Type'])
df.drop(['Provider Type'] ,axis=1, inplace=True)

EData= pd.concat([df, data_encoded ], axis=1)


In [8]:
data_encoded1 = pd.DataFrame (encoder.fit_transform(EData[['Primary Care Provider_PCP']]))
data_encoded1.columns = encoder.get_feature_names(['Primary Care Provider_PCP'])
EData.drop(['Primary Care Provider_PCP'] ,axis=1, inplace=True)

EData1 = pd.concat([EData, data_encoded1 ], axis=1)
EData1.head()

,Number of Certified Beds,Average Number of Residents Per Day,Overall Rating,Health Inspection Rating,QM Rating,Staffing Rating,Reported Total Nurse Staffing Hours per Resident per Day,Reported Physical Therapist Staffing Hours per Resident Per Day,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Number of Penalties,Avg Hospital Stay,Treatment cost (in thousands $),Hospital readmission rates(in %),Patient wait time (mins),Patient satisfaction rate(out of 10),ER wait time,Provider Type_Medicaid,Provider Type_Medicare,Provider Type_Medicare and Medicaid,Primary Care Provider_PCP_FP,Primary Care Provider_PCP_GER,Primary Care Provider_PCP_GIM,Primary Care Provider_PCP_GP,Primary Care Provider_PCP_No,Primary Care Provider_PCP_PD
0,92,78.0,4,3,5,3,3.45472,0.03404,40.667,0,4,0,0,3,20,32,32,4,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108,96.0,5,5,4,5,NaN,0.05574,3.333,0,0,0,0,5,4,42,15,5,5,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,88,75.7,2,2,4,3,3.23398,0.12765,104.000,1,4,0,0,3,14,36,46,3,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,60,49.9,5,5,4,3,3.10734,0.02423,14.400,0,0,0,0,4,5,21,32,6,4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60,49.3,2,2,4,2,3.17798,NaN,82.667,0,18,0,0,9,34,25,82,5,5,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
col = EData1.columns.tolist()
print(col)

['Number of Certified Beds', 'Average Number of Residents Per Day', 'Overall Rating', 'Health Inspection Rating', 'QM Rating', 'Staffing Rating', 'Reported Total Nurse Staffing Hours per Resident per Day', 'Reported Physical Therapist Staffing Hours per Resident Per Day', 'Total Weighted Health Survey Score', 'Number of Facility Reported Incidents', 'Number of Substantiated Complaints', 'Number of Fines', 'Total Number of Penalties', 'Avg Hospital Stay', 'Treatment cost (in thousands $)', 'Hospital readmission rates(in %)', 'Patient wait time (mins)', 'Patient satisfaction rate(out of 10)', 'ER wait time', 'Provider Type_Medicaid', 'Provider Type_Medicare', 'Provider Type_Medicare and Medicaid', 'Primary Care Provider_PCP_FP', 'Primary Care Provider_PCP_GER', 'Primary Care Provider_PCP_GIM', 'Primary Care Provider_PCP_GP', 'Primary Care Provider_PCP_No', 'Primary Care Provider_PCP_PD']


In [11]:
EData.head()

,Number of Certified Beds,Average Number of Residents Per Day,Overall Rating,Health Inspection Rating,QM Rating,Staffing Rating,Reported Total Nurse Staffing Hours per Resident per Day,Reported Physical Therapist Staffing Hours per Resident Per Day,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Number of Penalties,Avg Hospital Stay,Treatment cost (in thousands $),Hospital readmission rates(in %),Patient wait time (mins),Patient satisfaction rate(out of 10),ER wait time,Provider Type_Medicaid,Provider Type_Medicare,Provider Type_Medicare and Medicaid
0,92,78.0,4,3,5,3,3.45472,0.03404,40.667,0,4,0,0,3,20,32,32,4,4,0.0,0.0,1.0
1,108,96.0,5,5,4,5,NaN,0.05574,3.333,0,0,0,0,5,4,42,15,5,5,0.0,0.0,1.0
2,88,75.7,2,2,4,3,3.23398,0.12765,104.000,1,4,0,0,3,14,36,46,3,2,0.0,0.0,1.0
3,60,49.9,5,5,4,3,3.10734,0.02423,14.400,0,0,0,0,4,5,21,32,6,4,0.0,0.0,1.0
4,60,49.3,2,2,4,2,3.17798,NaN,82.667,0,18,0,0,9,34,25,82,5,5,0.0,0.0,1.0


**We are imputing the data in the next step using Iterative Imputer**

In [12]:
from fancyimpute import IterativeImputer as MICE

EData1 = MICE().fit_transform(EData1)
IData = pd.DataFrame(EData1 , columns=col)
IData.head()

,Number of Certified Beds,Average Number of Residents Per Day,Overall Rating,Health Inspection Rating,QM Rating,Staffing Rating,Reported Total Nurse Staffing Hours per Resident per Day,Reported Physical Therapist Staffing Hours per Resident Per Day,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Number of Penalties,Avg Hospital Stay,Treatment cost (in thousands $),Hospital readmission rates(in %),Patient wait time (mins),Patient satisfaction rate(out of 10),ER wait time,Provider Type_Medicaid,Provider Type_Medicare,Provider Type_Medicare and Medicaid,Primary Care Provider_PCP_FP,Primary Care Provider_PCP_GER,Primary Care Provider_PCP_GIM,Primary Care Provider_PCP_GP,Primary Care Provider_PCP_No,Primary Care Provider_PCP_PD
0,92.0,78.0,4.0,3.0,5.0,3.0,3.454720,0.034040,40.667,0.0,4.0,0.0,0.0,3.0,20.0,32.0,32.0,4.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108.0,96.0,5.0,5.0,4.0,5.0,4.816694,0.055740,3.333,0.0,0.0,0.0,0.0,5.0,4.0,42.0,15.0,5.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,88.0,75.7,2.0,2.0,4.0,3.0,3.233980,0.127650,104.000,1.0,4.0,0.0,0.0,3.0,14.0,36.0,46.0,3.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,60.0,49.9,5.0,5.0,4.0,3.0,3.107340,0.024230,14.400,0.0,0.0,0.0,0.0,4.0,5.0,21.0,32.0,6.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60.0,49.3,2.0,2.0,4.0,2.0,3.177980,0.094976,82.667,0.0,18.0,0.0,0.0,9.0,34.0,25.0,82.0,5.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


**Normalization/Standardization of the data**

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
NData=sc.fit_transform(IData)

In [16]:
df = pd.DataFrame(NData, columns=IData.columns)

In [17]:
df.to_csv('Provider_Info_Cleaned.csv')